# Entrenamiento del Modelo de Clasificación de Tickets

Este notebook documenta el proceso de entrenamiento del modelo de clasificación de tickets de soporte.

## 1. Importar librerías

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Cargar y explorar los datos

In [ ]:
# Cargar el dataset preprocesado
df = pd.read_csv("../data/sample_tickets.csv")

print(f"Forma del dataset: {df.shape}")
print(f"\nColumnas: {df.columns.tolist()}")
print(f"\nDistribución de categorías:")
print(df['category'].value_counts())

In [ ]:
# Visualizar la distribución de categorías
plt.figure(figsize=(10, 6))
df['category'].value_counts().plot(kind='bar', color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
plt.title('Distribución de Categorías en el Dataset')
plt.xlabel('Categoría')
plt.ylabel('Número de Tickets')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 3. Preparar los datos para el entrenamiento

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X = df['cleaned_description']
y = df['category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Tamaño del conjunto de entrenamiento: {len(X_train)}")
print(f"Tamaño del conjunto de prueba: {len(X_test)}")

## 4. Vectorización de texto con TF-IDF

In [ ]:
# Vectorización de texto con TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(f"Forma de la matriz TF-IDF de entrenamiento: {X_train_tfidf.shape}")
print(f"Forma de la matriz TF-IDF de prueba: {X_test_tfidf.shape}")

## 5. Entrenar el modelo

In [ ]:
# Entrenar el modelo de Regresión Logística
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_tfidf, y_train)

print("Modelo entrenado exitosamente")

## 6. Evaluar el modelo

In [ ]:
# Evaluar el modelo
y_pred = model.predict(X_test_tfidf)

metrics = {
    "accuracy": accuracy_score(y_test, y_pred),
    "precision": precision_score(y_test, y_pred, average="weighted"),
    "recall": recall_score(y_test, y_pred, average="weighted"),
    "f1_score": f1_score(y_test, y_pred, average="weighted")
}

print("Métricas del modelo:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
# Reporte de clasificación detallado
print("\nReporte de clasificación detallado:")
print(classification_report(y_test, y_pred))

## 7. Guardar el modelo

In [ ]:
# Guardar el modelo entrenado y el vectorizador
with open("../models/ticket_classifier.pkl", "wb") as f:
    pickle.dump({"model": model, "vectorizer": vectorizer, "metrics": metrics}, f)

print("Modelo y vectorizador guardados en ../models/ticket_classifier.pkl")

## 8. Ejemplos de predicción

In [ ]:
# Ejemplos de predicción
ejemplos = [
    "No puedo acceder a mi cuenta, olvidé mi contraseña",
    "Mi factura tiene un cargo incorrecto, necesito un reembolso",
    "El sistema se cuelga cuando intento cargar la página"
]

print("Ejemplos de predicción:")
for i, ejemplo in enumerate(ejemplos, 1):
    ejemplo_tfidf = vectorizer.transform([ejemplo])
    prediccion = model.predict(ejemplo_tfidf)[0]
    probabilidad = model.predict_proba(ejemplo_tfidf)[0].max()
    
    print(f"\nEjemplo {i}: {ejemplo}")
    print(f"Categoría predicha: {prediccion}")
    print(f"Probabilidad: {probabilidad:.3f}")